In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import scipy
import threading
import python_speech_features
from scipy.io import wavfile
import scipy.fftpack as fft
from scipy.signal import get_window
import IPython.display as ipd
import math
%matplotlib inline
import librosa
import pickle
from os import listdir
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN, LSTM
from keras.layers import Dropout
from os.path import isfile, join
from keras.layers import Flatten, Conv1D, Input, MaxPooling1D, Conv2D, MaxPooling2D, BatchNormalization, Reshape
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from tensorflow.keras.models import Model, load_model
from keras.layers import Input
from tensorflow.keras import layers
from tensorflow.keras import layers as L
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import backend as K
from tensorflow.keras import optimizers
import pickle
import os
from os import listdir
import pywt
from pywt import dwt, wavedec
from os.path import isfile, join
import torch
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [ ]:
dft = pd.read_csv('dataGendb4.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label

In [ ]:
less = 0
good = 0
window_size = 2048
hop_size =256
allLabels = []
sample_rate = 44100

def AttRNNSpeechModel(rnn_func=L.LSTM):
    inputs = Input(shape=(7, 20, 216))
    x = L.Conv2D(128, (5, 1), activation='relu', padding='same')(inputs)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(64, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(32, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(16, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(8, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(1, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)

    x = L.Lambda(lambda q: K.squeeze(q, -1), name='squeeze_last_dim')(x)

    x = L.Bidirectional(rnn_func(64, return_sequences=True)
                        )(x)
    x = L.Bidirectional(rnn_func(32, return_sequences=True)
                        )(x)

    xFirst = L.Lambda(lambda q: q[:, -1])(x)
    query = L.Dense(64)(xFirst)

    # dot product attention
    attScores = L.Dot(axes=[1, 2])([query, x])
    attScores = L.Softmax(name='attSoftmax')(attScores)

    # rescale sequence
    attVector = L.Dot(axes=[1, 1])([attScores, x])

    x = L.Dense(64, activation='relu')(attVector)
    x = L.Dense(32, activation='relu')(attVector)
    x = L.Dense(16)(x)

    output = L.Dense(84, activation='softmax', name='output')(x)

    model = Model(inputs=[inputs], outputs=[output])

    return model

model = AttRNNSpeechModel()#, rnn_func=L.LSTM)

model.compile(optimizer='adam', loss=['categorical_crossentropy'], metrics=['accuracy'])
model.summary()

In [ ]:
class CustomDataGen(tf.keras.utils.Sequence):
    
    def __init__(self, df,
                 batch_size, validation,
                 shuffle=True):

        self.df = df.copy()
        if validation:
            self.df = self.df[25000:]
        else:
            self.df = self.df[:25000]
        self.batch_size = batch_size
        self.shuffle = shuffle
        
        self.n = len(self.df)
    
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)
    
    def __get_input(self, path):
    
        objects = []
        with (open(path, "rb")) as openfile:
            while True:
                try:
                    objects.append(pickle.load(openfile))
                except EOFError:
                    break
        return objects[0]["audio"]

    
    def __get_output(self, tag):
        return lab[tag]
    
    def __get_data(self, batches):
        # Generates data containing batch_size samples

        files = batches["file"]
        tag = batches["tag"]

        X_batch = np.asarray([self.__get_input(x) for x in files])

        y_batch = np.asarray([self.__get_output(y) for y in tag])

        return X_batch, y_batch
    
    def __getitem__(self, index):
        
        batches = self.df[index * self.batch_size:(index + 1) * self.batch_size]
        X, y = self.__get_data(batches)        
        return X, y
    
    def __len__(self):
        return self.n // self.batch_size

In [ ]:

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.00001) 
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
mc = ModelCheckpoint('best_model.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
history=model.fit_generator(generator=training_generator, validation_data = validation_generator, epochs=50, callbacks=[mc,es], use_multiprocessing=True, workers=6)

In [ ]:
dft = pd.read_csv('dataGendb10.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
less = 0
good = 0
window_size = 2048
hop_size =256
allLabels = []
sample_rate = 44100

def AttRNNSpeechModel(rnn_func=L.LSTM):
    inputs = Input(shape=(7, 20, 216))
    x = L.Conv2D(128, (5, 1), activation='relu', padding='same')(inputs)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(64, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(32, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(16, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(8, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(1, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)

    x = L.Lambda(lambda q: K.squeeze(q, -1), name='squeeze_last_dim')(x)

    x = L.Bidirectional(rnn_func(64, return_sequences=True)
                        )(x)
    x = L.Bidirectional(rnn_func(32, return_sequences=True)
                        )(x)

    xFirst = L.Lambda(lambda q: q[:, -1])(x)
    query = L.Dense(64)(xFirst)

    # dot product attention
    attScores = L.Dot(axes=[1, 2])([query, x])
    attScores = L.Softmax(name='attSoftmax')(attScores)

    # rescale sequence
    attVector = L.Dot(axes=[1, 1])([attScores, x])

    x = L.Dense(64, activation='relu')(attVector)
    x = L.Dense(32, activation='relu')(attVector)
    x = L.Dense(16)(x)

    output = L.Dense(84, activation='softmax', name='output')(x)

    model = Model(inputs=[inputs], outputs=[output])

    return model

model = AttRNNSpeechModel()#, rnn_func=L.LSTM)

model.compile(optimizer='adam', loss=['categorical_crossentropy'], metrics=['accuracy'])
model.summary()

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.00001) 
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
mc = ModelCheckpoint('best_modeldb10.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
history=model.fit_generator(generator=training_generator, validation_data = validation_generator, epochs=50, callbacks=[mc,es], use_multiprocessing=True, workers=6)

In [ ]:
dft = pd.read_csv('dataGendb12.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
less = 0
good = 0
window_size = 2048
hop_size =256
allLabels = []
sample_rate = 44100

def AttRNNSpeechModel(rnn_func=L.LSTM):
    inputs = Input(shape=(7, 20, 216))
    x = L.Conv2D(128, (5, 1), activation='relu', padding='same')(inputs)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(64, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(32, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(16, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(8, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(1, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)

    x = L.Lambda(lambda q: K.squeeze(q, -1), name='squeeze_last_dim')(x)

    x = L.Bidirectional(rnn_func(64, return_sequences=True)
                        )(x)
    x = L.Bidirectional(rnn_func(32, return_sequences=True)
                        )(x)

    xFirst = L.Lambda(lambda q: q[:, -1])(x)
    query = L.Dense(64)(xFirst)

    # dot product attention
    attScores = L.Dot(axes=[1, 2])([query, x])
    attScores = L.Softmax(name='attSoftmax')(attScores)

    # rescale sequence
    attVector = L.Dot(axes=[1, 1])([attScores, x])

    x = L.Dense(64, activation='relu')(attVector)
    x = L.Dense(32, activation='relu')(attVector)
    x = L.Dense(16)(x)

    output = L.Dense(84, activation='softmax', name='output')(x)

    model = Model(inputs=[inputs], outputs=[output])

    return model

model = AttRNNSpeechModel()#, rnn_func=L.LSTM)

model.compile(optimizer='adam', loss=['categorical_crossentropy'], metrics=['accuracy'])
model.summary()

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.00001) 
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
mc = ModelCheckpoint('best_modeldb12.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
history=model.fit_generator(generator=training_generator, validation_data = validation_generator, epochs=50, callbacks=[mc,es], use_multiprocessing=True, workers=6)

In [ ]:
dft = pd.read_csv('dataGensym4.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
less = 0
good = 0
window_size = 2048
hop_size =256
allLabels = []
sample_rate = 44100

def AttRNNSpeechModel(rnn_func=L.LSTM):
    inputs = Input(shape=(7, 20, 216))
    x = L.Conv2D(128, (5, 1), activation='relu', padding='same')(inputs)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(64, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(32, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(16, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(8, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(1, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)

    x = L.Lambda(lambda q: K.squeeze(q, -1), name='squeeze_last_dim')(x)

    x = L.Bidirectional(rnn_func(64, return_sequences=True)
                        )(x)
    x = L.Bidirectional(rnn_func(32, return_sequences=True)
                        )(x)

    xFirst = L.Lambda(lambda q: q[:, -1])(x)
    query = L.Dense(64)(xFirst)

    # dot product attention
    attScores = L.Dot(axes=[1, 2])([query, x])
    attScores = L.Softmax(name='attSoftmax')(attScores)

    # rescale sequence
    attVector = L.Dot(axes=[1, 1])([attScores, x])

    x = L.Dense(64, activation='relu')(attVector)
    x = L.Dense(32, activation='relu')(attVector)
    x = L.Dense(16)(x)

    output = L.Dense(84, activation='softmax', name='output')(x)

    model = Model(inputs=[inputs], outputs=[output])

    return model

model = AttRNNSpeechModel()#, rnn_func=L.LSTM)

model.compile(optimizer='adam', loss=['categorical_crossentropy'], metrics=['accuracy'])
model.summary()

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.00001) 
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
mc = ModelCheckpoint('best_modelsym4.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
history=model.fit_generator(generator=training_generator, validation_data = validation_generator, epochs=50, callbacks=[mc,es], use_multiprocessing=True, workers=6)

In [ ]:
dft = pd.read_csv('dataGensym6.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
less = 0
good = 0
window_size = 2048
hop_size =256
allLabels = []
sample_rate = 44100

def AttRNNSpeechModel(rnn_func=L.LSTM):
    inputs = Input(shape=(7, 20, 216))
    x = L.Conv2D(128, (5, 1), activation='relu', padding='same')(inputs)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(64, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(32, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(16, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(8, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(1, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)

    x = L.Lambda(lambda q: K.squeeze(q, -1), name='squeeze_last_dim')(x)

    x = L.Bidirectional(rnn_func(64, return_sequences=True)
                        )(x)
    x = L.Bidirectional(rnn_func(32, return_sequences=True)
                        )(x)

    xFirst = L.Lambda(lambda q: q[:, -1])(x)
    query = L.Dense(64)(xFirst)

    # dot product attention
    attScores = L.Dot(axes=[1, 2])([query, x])
    attScores = L.Softmax(name='attSoftmax')(attScores)

    # rescale sequence
    attVector = L.Dot(axes=[1, 1])([attScores, x])

    x = L.Dense(64, activation='relu')(attVector)
    x = L.Dense(32, activation='relu')(attVector)
    x = L.Dense(16)(x)

    output = L.Dense(84, activation='softmax', name='output')(x)

    model = Model(inputs=[inputs], outputs=[output])

    return model

model = AttRNNSpeechModel()#, rnn_func=L.LSTM)

model.compile(optimizer='adam', loss=['categorical_crossentropy'], metrics=['accuracy'])
model.summary()

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.00001) 
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
mc = ModelCheckpoint('best_modelsym6.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
history=model.fit_generator(generator=training_generator, validation_data = validation_generator, epochs=50, callbacks=[mc,es], use_multiprocessing=True, workers=6)

In [ ]:
dft = pd.read_csv('dataGensym9.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
less = 0
good = 0
window_size = 2048
hop_size =256
allLabels = []
sample_rate = 44100

def AttRNNSpeechModel(rnn_func=L.LSTM):
    inputs = Input(shape=(7, 20, 216))
    x = L.Conv2D(128, (5, 1), activation='relu', padding='same')(inputs)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(64, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(32, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(16, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(8, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(1, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)

    x = L.Lambda(lambda q: K.squeeze(q, -1), name='squeeze_last_dim')(x)

    x = L.Bidirectional(rnn_func(64, return_sequences=True)
                        )(x)
    x = L.Bidirectional(rnn_func(32, return_sequences=True)
                        )(x)

    xFirst = L.Lambda(lambda q: q[:, -1])(x)
    query = L.Dense(64)(xFirst)

    # dot product attention
    attScores = L.Dot(axes=[1, 2])([query, x])
    attScores = L.Softmax(name='attSoftmax')(attScores)

    # rescale sequence
    attVector = L.Dot(axes=[1, 1])([attScores, x])

    x = L.Dense(64, activation='relu')(attVector)
    x = L.Dense(32, activation='relu')(attVector)
    x = L.Dense(16)(x)

    output = L.Dense(84, activation='softmax', name='output')(x)

    model = Model(inputs=[inputs], outputs=[output])

    return model

model = AttRNNSpeechModel()#, rnn_func=L.LSTM)

model.compile(optimizer='adam', loss=['categorical_crossentropy'], metrics=['accuracy'])
model.summary()

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.00001) 
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
mc = ModelCheckpoint('best_modelsym9.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
history=model.fit_generator(generator=training_generator, validation_data = validation_generator, epochs=50, callbacks=[mc,es], use_multiprocessing=True, workers=6)

In [ ]:
dft = pd.read_csv('dataGensym16.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
less = 0
good = 0
window_size = 2048
hop_size =256
allLabels = []
sample_rate = 44100

def AttRNNSpeechModel(rnn_func=L.LSTM):
    inputs = Input(shape=(7, 20, 216))
    x = L.Conv2D(128, (5, 1), activation='relu', padding='same')(inputs)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(64, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(32, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(16, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(8, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(1, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)

    x = L.Lambda(lambda q: K.squeeze(q, -1), name='squeeze_last_dim')(x)

    x = L.Bidirectional(rnn_func(64, return_sequences=True)
                        )(x)
    x = L.Bidirectional(rnn_func(32, return_sequences=True)
                        )(x)

    xFirst = L.Lambda(lambda q: q[:, -1])(x)
    query = L.Dense(64)(xFirst)

    # dot product attention
    attScores = L.Dot(axes=[1, 2])([query, x])
    attScores = L.Softmax(name='attSoftmax')(attScores)

    # rescale sequence
    attVector = L.Dot(axes=[1, 1])([attScores, x])

    x = L.Dense(64, activation='relu')(attVector)
    x = L.Dense(32, activation='relu')(attVector)
    x = L.Dense(16)(x)

    output = L.Dense(84, activation='softmax', name='output')(x)

    model = Model(inputs=[inputs], outputs=[output])

    return model

model = AttRNNSpeechModel()#, rnn_func=L.LSTM)

model.compile(optimizer='adam', loss=['categorical_crossentropy'], metrics=['accuracy'])
model.summary()

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.00001) 
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
mc = ModelCheckpoint('best_modelsym16.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
history=model.fit_generator(generator=training_generator, validation_data = validation_generator, epochs=50, callbacks=[mc,es], use_multiprocessing=True, workers=6)

In [ ]:
dft = pd.read_csv('dataGencoif3.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
less = 0
good = 0
window_size = 2048
hop_size =256
allLabels = []
sample_rate = 44100

def AttRNNSpeechModel(rnn_func=L.LSTM):
    inputs = Input(shape=(7, 20, 216))
    x = L.Conv2D(128, (5, 1), activation='relu', padding='same')(inputs)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(64, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(32, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(16, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(8, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(1, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)

    x = L.Lambda(lambda q: K.squeeze(q, -1), name='squeeze_last_dim')(x)

    x = L.Bidirectional(rnn_func(64, return_sequences=True)
                        )(x)
    x = L.Bidirectional(rnn_func(32, return_sequences=True)
                        )(x)

    xFirst = L.Lambda(lambda q: q[:, -1])(x)
    query = L.Dense(64)(xFirst)

    # dot product attention
    attScores = L.Dot(axes=[1, 2])([query, x])
    attScores = L.Softmax(name='attSoftmax')(attScores)

    # rescale sequence
    attVector = L.Dot(axes=[1, 1])([attScores, x])

    x = L.Dense(64, activation='relu')(attVector)
    x = L.Dense(32, activation='relu')(attVector)
    x = L.Dense(16)(x)

    output = L.Dense(84, activation='softmax', name='output')(x)

    model = Model(inputs=[inputs], outputs=[output])

    return model

model = AttRNNSpeechModel()#, rnn_func=L.LSTM)

model.compile(optimizer='adam', loss=['categorical_crossentropy'], metrics=['accuracy'])
model.summary()

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.00001) 
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
mc = ModelCheckpoint('best_modelcoif3.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
history=model.fit_generator(generator=training_generator, validation_data = validation_generator, epochs=50, callbacks=[mc,es], use_multiprocessing=True, workers=6)

In [ ]:
dft = pd.read_csv('dataGencoif6.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
less = 0
good = 0
window_size = 2048
hop_size =256
allLabels = []
sample_rate = 44100

def AttRNNSpeechModel(rnn_func=L.LSTM):
    inputs = Input(shape=(7, 20, 216))
    x = L.Conv2D(128, (5, 1), activation='relu', padding='same')(inputs)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(64, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(32, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(16, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(8, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(1, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)

    x = L.Lambda(lambda q: K.squeeze(q, -1), name='squeeze_last_dim')(x)

    x = L.Bidirectional(rnn_func(64, return_sequences=True)
                        )(x)
    x = L.Bidirectional(rnn_func(32, return_sequences=True)
                        )(x)

    xFirst = L.Lambda(lambda q: q[:, -1])(x)
    query = L.Dense(64)(xFirst)

    # dot product attention
    attScores = L.Dot(axes=[1, 2])([query, x])
    attScores = L.Softmax(name='attSoftmax')(attScores)

    # rescale sequence
    attVector = L.Dot(axes=[1, 1])([attScores, x])

    x = L.Dense(64, activation='relu')(attVector)
    x = L.Dense(32, activation='relu')(attVector)
    x = L.Dense(16)(x)

    output = L.Dense(84, activation='softmax', name='output')(x)

    model = Model(inputs=[inputs], outputs=[output])

    return model

model = AttRNNSpeechModel()#, rnn_func=L.LSTM)

model.compile(optimizer='adam', loss=['categorical_crossentropy'], metrics=['accuracy'])
model.summary()

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.00001) 
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
mc = ModelCheckpoint('best_modelcoif6.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
history=model.fit_generator(generator=training_generator, validation_data = validation_generator, epochs=50, callbacks=[mc,es], use_multiprocessing=True, workers=6)

In [ ]:
dft = pd.read_csv('dataGencoif8.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
less = 0
good = 0
window_size = 2048
hop_size =256
allLabels = []
sample_rate = 44100

def AttRNNSpeechModel(rnn_func=L.LSTM):
    inputs = Input(shape=(7, 20, 216))
    x = L.Conv2D(128, (5, 1), activation='relu', padding='same')(inputs)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(64, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(32, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(16, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(8, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(1, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)

    x = L.Lambda(lambda q: K.squeeze(q, -1), name='squeeze_last_dim')(x)

    x = L.Bidirectional(rnn_func(64, return_sequences=True)
                        )(x)
    x = L.Bidirectional(rnn_func(32, return_sequences=True)
                        )(x)

    xFirst = L.Lambda(lambda q: q[:, -1])(x)
    query = L.Dense(64)(xFirst)

    # dot product attention
    attScores = L.Dot(axes=[1, 2])([query, x])
    attScores = L.Softmax(name='attSoftmax')(attScores)

    # rescale sequence
    attVector = L.Dot(axes=[1, 1])([attScores, x])

    x = L.Dense(64, activation='relu')(attVector)
    x = L.Dense(32, activation='relu')(attVector)
    x = L.Dense(16)(x)

    output = L.Dense(84, activation='softmax', name='output')(x)

    model = Model(inputs=[inputs], outputs=[output])

    return model

model = AttRNNSpeechModel()#, rnn_func=L.LSTM)

model.compile(optimizer='adam', loss=['categorical_crossentropy'], metrics=['accuracy'])
model.summary()

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.00001) 
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
mc = ModelCheckpoint('best_modelcoif8.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
history=model.fit_generator(generator=training_generator, validation_data = validation_generator, epochs=50, callbacks=[mc,es], use_multiprocessing=True, workers=6)

In [ ]:
dft = pd.read_csv('dataGencoif10.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
less = 0
good = 0
window_size = 2048
hop_size =256
allLabels = []
sample_rate = 44100

def AttRNNSpeechModel(rnn_func=L.LSTM):
    inputs = Input(shape=(7, 20, 216))
    x = L.Conv2D(128, (5, 1), activation='relu', padding='same')(inputs)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(64, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(32, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(16, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(8, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(1, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)

    x = L.Lambda(lambda q: K.squeeze(q, -1), name='squeeze_last_dim')(x)

    x = L.Bidirectional(rnn_func(64, return_sequences=True)
                        )(x)
    x = L.Bidirectional(rnn_func(32, return_sequences=True)
                        )(x)

    xFirst = L.Lambda(lambda q: q[:, -1])(x)
    query = L.Dense(64)(xFirst)

    # dot product attention
    attScores = L.Dot(axes=[1, 2])([query, x])
    attScores = L.Softmax(name='attSoftmax')(attScores)

    # rescale sequence
    attVector = L.Dot(axes=[1, 1])([attScores, x])

    x = L.Dense(64, activation='relu')(attVector)
    x = L.Dense(32, activation='relu')(attVector)
    x = L.Dense(16)(x)

    output = L.Dense(84, activation='softmax', name='output')(x)

    model = Model(inputs=[inputs], outputs=[output])

    return model

model = AttRNNSpeechModel()#, rnn_func=L.LSTM)

model.compile(optimizer='adam', loss=['categorical_crossentropy'], metrics=['accuracy'])
model.summary()

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.00001) 
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
mc = ModelCheckpoint('best_modelcoif10.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
history=model.fit_generator(generator=training_generator, validation_data = validation_generator, epochs=50, callbacks=[mc,es], use_multiprocessing=True, workers=6)

In [ ]:
dft = pd.read_csv('dataGenbior1.1.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
less = 0
good = 0
window_size = 2048
hop_size =256
allLabels = []
sample_rate = 44100

def AttRNNSpeechModel(rnn_func=L.LSTM):
    inputs = Input(shape=(7, 20, 216))
    x = L.Conv2D(128, (5, 1), activation='relu', padding='same')(inputs)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(64, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(32, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(16, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(8, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(1, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)

    x = L.Lambda(lambda q: K.squeeze(q, -1), name='squeeze_last_dim')(x)

    x = L.Bidirectional(rnn_func(64, return_sequences=True)
                        )(x)
    x = L.Bidirectional(rnn_func(32, return_sequences=True)
                        )(x)

    xFirst = L.Lambda(lambda q: q[:, -1])(x)
    query = L.Dense(64)(xFirst)

    # dot product attention
    attScores = L.Dot(axes=[1, 2])([query, x])
    attScores = L.Softmax(name='attSoftmax')(attScores)

    # rescale sequence
    attVector = L.Dot(axes=[1, 1])([attScores, x])

    x = L.Dense(64, activation='relu')(attVector)
    x = L.Dense(32, activation='relu')(attVector)
    x = L.Dense(16)(x)

    output = L.Dense(84, activation='softmax', name='output')(x)

    model = Model(inputs=[inputs], outputs=[output])

    return model

model = AttRNNSpeechModel()#, rnn_func=L.LSTM)

model.compile(optimizer='adam', loss=['categorical_crossentropy'], metrics=['accuracy'])
model.summary()

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.00001) 
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
mc = ModelCheckpoint('best_modelbior1.1.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
history=model.fit_generator(generator=training_generator, validation_data = validation_generator, epochs=50, callbacks=[mc,es], use_multiprocessing=True, workers=6)

In [ ]:
dft = pd.read_csv('dataGenbior1.5.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
less = 0
good = 0
window_size = 2048
hop_size =256
allLabels = []
sample_rate = 44100

def AttRNNSpeechModel(rnn_func=L.LSTM):
    inputs = Input(shape=(7, 20, 216))
    x = L.Conv2D(128, (5, 1), activation='relu', padding='same')(inputs)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(64, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(32, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(16, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(8, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(1, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)

    x = L.Lambda(lambda q: K.squeeze(q, -1), name='squeeze_last_dim')(x)

    x = L.Bidirectional(rnn_func(64, return_sequences=True)
                        )(x)
    x = L.Bidirectional(rnn_func(32, return_sequences=True)
                        )(x)

    xFirst = L.Lambda(lambda q: q[:, -1])(x)
    query = L.Dense(64)(xFirst)

    # dot product attention
    attScores = L.Dot(axes=[1, 2])([query, x])
    attScores = L.Softmax(name='attSoftmax')(attScores)

    # rescale sequence
    attVector = L.Dot(axes=[1, 1])([attScores, x])

    x = L.Dense(64, activation='relu')(attVector)
    x = L.Dense(32, activation='relu')(attVector)
    x = L.Dense(16)(x)

    output = L.Dense(84, activation='softmax', name='output')(x)

    model = Model(inputs=[inputs], outputs=[output])

    return model

model = AttRNNSpeechModel()#, rnn_func=L.LSTM)

model.compile(optimizer='adam', loss=['categorical_crossentropy'], metrics=['accuracy'])
model.summary()

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.00001) 
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
mc = ModelCheckpoint('best_modelbior1.5.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
history=model.fit_generator(generator=training_generator, validation_data = validation_generator, epochs=50, callbacks=[mc,es], use_multiprocessing=True, workers=6)

In [ ]:
dft = pd.read_csv('dataGenbior3.5.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
less = 0
good = 0
window_size = 2048
hop_size =256
allLabels = []
sample_rate = 44100

def AttRNNSpeechModel(rnn_func=L.LSTM):
    inputs = Input(shape=(7, 20, 216))
    x = L.Conv2D(128, (5, 1), activation='relu', padding='same')(inputs)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(64, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(32, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(16, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(8, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(1, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)

    x = L.Lambda(lambda q: K.squeeze(q, -1), name='squeeze_last_dim')(x)

    x = L.Bidirectional(rnn_func(64, return_sequences=True)
                        )(x)
    x = L.Bidirectional(rnn_func(32, return_sequences=True)
                        )(x)

    xFirst = L.Lambda(lambda q: q[:, -1])(x)
    query = L.Dense(64)(xFirst)

    # dot product attention
    attScores = L.Dot(axes=[1, 2])([query, x])
    attScores = L.Softmax(name='attSoftmax')(attScores)

    # rescale sequence
    attVector = L.Dot(axes=[1, 1])([attScores, x])

    x = L.Dense(64, activation='relu')(attVector)
    x = L.Dense(32, activation='relu')(attVector)
    x = L.Dense(16)(x)

    output = L.Dense(84, activation='softmax', name='output')(x)

    model = Model(inputs=[inputs], outputs=[output])

    return model

model = AttRNNSpeechModel()#, rnn_func=L.LSTM)

model.compile(optimizer='adam', loss=['categorical_crossentropy'], metrics=['accuracy'])
model.summary()

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.00001) 
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
mc = ModelCheckpoint('best_modelbior2.4.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
history=model.fit_generator(generator=training_generator, validation_data = validation_generator, epochs=50, callbacks=[mc,es], use_multiprocessing=True, workers=6)

In [ ]:
dft = pd.read_csv('dataGenbior3.5.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
less = 0
good = 0
window_size = 2048
hop_size =256
allLabels = []
sample_rate = 44100

def AttRNNSpeechModel(rnn_func=L.LSTM):
    inputs = Input(shape=(7, 20, 216))
    x = L.Conv2D(128, (5, 1), activation='relu', padding='same')(inputs)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(64, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(32, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(16, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(8, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(1, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)

    x = L.Lambda(lambda q: K.squeeze(q, -1), name='squeeze_last_dim')(x)

    x = L.Bidirectional(rnn_func(64, return_sequences=True)
                        )(x)
    x = L.Bidirectional(rnn_func(32, return_sequences=True)
                        )(x)

    xFirst = L.Lambda(lambda q: q[:, -1])(x)
    query = L.Dense(64)(xFirst)

    # dot product attention
    attScores = L.Dot(axes=[1, 2])([query, x])
    attScores = L.Softmax(name='attSoftmax')(attScores)

    # rescale sequence
    attVector = L.Dot(axes=[1, 1])([attScores, x])

    x = L.Dense(64, activation='relu')(attVector)
    x = L.Dense(32, activation='relu')(attVector)
    x = L.Dense(16)(x)

    output = L.Dense(84, activation='softmax', name='output')(x)

    model = Model(inputs=[inputs], outputs=[output])

    return model

model = AttRNNSpeechModel()#, rnn_func=L.LSTM)

model.compile(optimizer='adam', loss=['categorical_crossentropy'], metrics=['accuracy'])
model.summary()

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.00001) 
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
mc = ModelCheckpoint('best_modelbior3.5.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
history=model.fit_generator(generator=training_generator, validation_data = validation_generator, epochs=50, callbacks=[mc,es], use_multiprocessing=True, workers=6)

In [ ]:
dft = pd.read_csv('dataGenrbio1.3.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
less = 0
good = 0
window_size = 2048
hop_size =256
allLabels = []
sample_rate = 44100

def AttRNNSpeechModel(rnn_func=L.LSTM):
    inputs = Input(shape=(7, 20, 216))
    x = L.Conv2D(128, (5, 1), activation='relu', padding='same')(inputs)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(64, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(32, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(16, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(8, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(1, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)

    x = L.Lambda(lambda q: K.squeeze(q, -1), name='squeeze_last_dim')(x)

    x = L.Bidirectional(rnn_func(64, return_sequences=True)
                        )(x)
    x = L.Bidirectional(rnn_func(32, return_sequences=True)
                        )(x)

    xFirst = L.Lambda(lambda q: q[:, -1])(x)
    query = L.Dense(64)(xFirst)

    # dot product attention
    attScores = L.Dot(axes=[1, 2])([query, x])
    attScores = L.Softmax(name='attSoftmax')(attScores)

    # rescale sequence
    attVector = L.Dot(axes=[1, 1])([attScores, x])

    x = L.Dense(64, activation='relu')(attVector)
    x = L.Dense(32, activation='relu')(attVector)
    x = L.Dense(16)(x)

    output = L.Dense(84, activation='softmax', name='output')(x)

    model = Model(inputs=[inputs], outputs=[output])

    return model

model = AttRNNSpeechModel()#, rnn_func=L.LSTM)

model.compile(optimizer='adam', loss=['categorical_crossentropy'], metrics=['accuracy'])
model.summary()

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.00001) 
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
mc = ModelCheckpoint('best_modelrbio1.3.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
history=model.fit_generator(generator=training_generator, validation_data = validation_generator, epochs=50, callbacks=[mc,es], use_multiprocessing=True, workers=6)

In [ ]:
dft = pd.read_csv('dataGenrbio1.5.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
less = 0
good = 0
window_size = 2048
hop_size =256
allLabels = []
sample_rate = 44100

def AttRNNSpeechModel(rnn_func=L.LSTM):
    inputs = Input(shape=(7, 20, 216))
    x = L.Conv2D(128, (5, 1), activation='relu', padding='same')(inputs)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(64, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(32, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(16, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(8, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(1, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)

    x = L.Lambda(lambda q: K.squeeze(q, -1), name='squeeze_last_dim')(x)

    x = L.Bidirectional(rnn_func(64, return_sequences=True)
                        )(x)
    x = L.Bidirectional(rnn_func(32, return_sequences=True)
                        )(x)

    xFirst = L.Lambda(lambda q: q[:, -1])(x)
    query = L.Dense(64)(xFirst)

    # dot product attention
    attScores = L.Dot(axes=[1, 2])([query, x])
    attScores = L.Softmax(name='attSoftmax')(attScores)

    # rescale sequence
    attVector = L.Dot(axes=[1, 1])([attScores, x])

    x = L.Dense(64, activation='relu')(attVector)
    x = L.Dense(32, activation='relu')(attVector)
    x = L.Dense(16)(x)

    output = L.Dense(84, activation='softmax', name='output')(x)

    model = Model(inputs=[inputs], outputs=[output])

    return model

model = AttRNNSpeechModel()#, rnn_func=L.LSTM)

model.compile(optimizer='adam', loss=['categorical_crossentropy'], metrics=['accuracy'])
model.summary()

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.00001) 
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
mc = ModelCheckpoint('best_modelrbio1.5.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
history=model.fit_generator(generator=training_generator, validation_data = validation_generator, epochs=50, callbacks=[mc,es], use_multiprocessing=True, workers=6)

In [ ]:
dft = pd.read_csv('dataGenrbio2.4.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
less = 0
good = 0
window_size = 2048
hop_size =256
allLabels = []
sample_rate = 44100

def AttRNNSpeechModel(rnn_func=L.LSTM):
    inputs = Input(shape=(7, 20, 216))
    x = L.Conv2D(128, (5, 1), activation='relu', padding='same')(inputs)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(64, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(32, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(16, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(8, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(1, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)

    x = L.Lambda(lambda q: K.squeeze(q, -1), name='squeeze_last_dim')(x)

    x = L.Bidirectional(rnn_func(64, return_sequences=True)
                        )(x)
    x = L.Bidirectional(rnn_func(32, return_sequences=True)
                        )(x)

    xFirst = L.Lambda(lambda q: q[:, -1])(x)
    query = L.Dense(64)(xFirst)

    # dot product attention
    attScores = L.Dot(axes=[1, 2])([query, x])
    attScores = L.Softmax(name='attSoftmax')(attScores)

    # rescale sequence
    attVector = L.Dot(axes=[1, 1])([attScores, x])

    x = L.Dense(64, activation='relu')(attVector)
    x = L.Dense(32, activation='relu')(attVector)
    x = L.Dense(16)(x)

    output = L.Dense(84, activation='softmax', name='output')(x)

    model = Model(inputs=[inputs], outputs=[output])

    return model

model = AttRNNSpeechModel()#, rnn_func=L.LSTM)

model.compile(optimizer='adam', loss=['categorical_crossentropy'], metrics=['accuracy'])
model.summary()

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.00001) 
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
mc = ModelCheckpoint('best_modelrbio2.4.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
history=model.fit_generator(generator=training_generator, validation_data = validation_generator, epochs=50, callbacks=[mc,es], use_multiprocessing=True, workers=6)

In [ ]:
dft = pd.read_csv('dataGenrbio6.8.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
less = 0
good = 0
window_size = 2048
hop_size =256
allLabels = []
sample_rate = 44100

def AttRNNSpeechModel(rnn_func=L.LSTM):
    inputs = Input(shape=(7, 20, 216))
    x = L.Conv2D(128, (5, 1), activation='relu', padding='same')(inputs)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(64, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(32, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(16, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(8, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(1, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)

    x = L.Lambda(lambda q: K.squeeze(q, -1), name='squeeze_last_dim')(x)

    x = L.Bidirectional(rnn_func(64, return_sequences=True)
                        )(x)
    x = L.Bidirectional(rnn_func(32, return_sequences=True)
                        )(x)

    xFirst = L.Lambda(lambda q: q[:, -1])(x)
    query = L.Dense(64)(xFirst)

    # dot product attention
    attScores = L.Dot(axes=[1, 2])([query, x])
    attScores = L.Softmax(name='attSoftmax')(attScores)

    # rescale sequence
    attVector = L.Dot(axes=[1, 1])([attScores, x])

    x = L.Dense(64, activation='relu')(attVector)
    x = L.Dense(32, activation='relu')(attVector)
    x = L.Dense(16)(x)

    output = L.Dense(84, activation='softmax', name='output')(x)

    model = Model(inputs=[inputs], outputs=[output])

    return model

model = AttRNNSpeechModel()#, rnn_func=L.LSTM)

model.compile(optimizer='adam', loss=['categorical_crossentropy'], metrics=['accuracy'])
model.summary()

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.00001) 
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
mc = ModelCheckpoint('best_modelrbio6.8.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
history=model.fit_generator(generator=training_generator, validation_data = validation_generator, epochs=50, callbacks=[mc,es], use_multiprocessing=True, workers=6)